In [7]:
# input group as dataframe with position and directon vectors as two columns. 

import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
import cv2
import functools
import math


# check dataframe for incompatible data 
def check_data(df, epsilon = .000001):

    for i in range(len(df.index)):
        
        sum_dir = np.linalg.norm([df.loc[i, "x_dir"],df.loc[i, "y_dir"]])

        if (abs(1 - sum_dir) > epsilon):
            
            raise Exception("The direction vector at index " + str(i) + " is not a unit vector.")

            
# returns the centroid of points in a dataframe. If periodic is true, returns the ciruclar mean. The centroid is a 1x2 tuple.
def get_centroid(df, precision = 6, periodic = (False, None,None)):

    if periodic[0] == False:
        xmean = df.loc[:,'x_pos'].mean()
        ymean = df.loc[:,'y_pos'].mean()
        x = round(xmean, precision)
        y = round(ymean, precision)
        return((x,y))
    
    
    elif periodic[0] == True:
        circlemeanx = mean_of_circular_vals(df['x_pos'], periodic[1])
        circlemeany = mean_of_circular_vals(df['y_pos'], periodic[2])
        x = round(circlemeanx, precision)
        y = round(circlemeany, precision)
        return((x,y))

# returns the average distance all individuals in the dataframe are from the centroid as a scalar.
def average_distance_from_centroid(df, precision = 6, periodic = (False,None,None)):
    
    
    disp_x, disp_y = __displacement_from_centroid__(df, periodic)
    
    n = len(disp_x)
    
    sum_dist = sum(np.linalg.norm([disp_x[i], disp_y[i]]) for i in range(n))
    
    average_dist = round((sum_dist / n) , precision)
    
    return (average_dist)
    
# returns the average direction of individuals in the dataframe as a 1x2 tuple.
def group_average_direction(df, precision = 6): 
    
    x_dir = round(df.loc[:,'x_dir'].mean(), precision)
    y_dir = round(df.loc[:,'y_dir'].mean(), precision)
    
    return((x_dir, y_dir))

# returns the polarization value [0,1] of the individuals in the dataframe. See Couzin et al, "Collective Memory...", 2002
def group_polarization(df, precision = 6):
    
    n = (len(df.index))
    x_dir =  df.loc[:,'x_dir'].sum()
    y_dir = df.loc[:,'y_dir'].sum()
    magnitude = np.linalg.norm([x_dir, y_dir])
    return(round((magnitude / n), precision))
    
# returns the angular momentum value [0,1] of the individuals around the centroid
# the dataframe. See Couzin et al, "Collective Memory...", 2002
def group_angular_momentum(df, precision = 6, periodic = (False, None, None)):

    n = (len(df.index))
    disp_x, disp_y = __displacement_from_centroid__(df, periodic)
    cross = sum(np.cross([disp_x[i],disp_y[i]], [df.loc[i, 'x_dir'],df.loc[i, 'y_dir']]) for i in range(n))
    magnitude = np.linalg.norm(cross)

    return(round((magnitude / n), precision))
    
# private method to calculate each indivudal's displacement from the centroid.
def __displacement_from_centroid__(df, periodic = (False,None,None)):
    
    if periodic[0] == False: 
        
        centroid = get_centroid(df, periodic = periodic)

        disp_x = df.loc[:,'x_pos'] - centroid[0]
        disp_y = df.loc[:,'y_pos'] - centroid[1]
    
    
    elif periodic[0] == True: 
        
       
        matrix_data = df[['x_pos','y_pos']].to_numpy()
        centroid = get_centroid(df, periodic = periodic)
        
        
        vectors = list(map(functools.partial(vector_between_periodic_points, point_y = centroid, arena_size_x = periodic[1], 
                                             arena_size_y = periodic[2]), matrix_data))
        
        vectors = np.array(vectors)
        disp_x = vectors[:,0]
        disp_y = vectors[:,1]
    
    return(disp_x,disp_y)


# returns a dictionary with the all statistcs for a dataframe. 
def summary_statistics(df, precision = 6, periodic = [False,None,None]):
    
    centroid = get_centroid(df, precision, periodic)
    
    average_distance = average_distance_from_centroid(df, precision,periodic)
    
    average_dir = group_average_direction(df, precision)

    polarization = group_polarization(df, precision)

    angular_momentum = group_angular_momentum(df, precision, periodic)
 
    number = len(df)
    
    stats = {'centroid' : centroid, 'average_distance': average_distance, 
            'average_dir': average_dir, 'polarization': polarization, 'angular_momentum': angular_momentum,
            'number': number}
    
    return(stats)

# gets labels from DBSCAN. If periodic = True, DBSCAN is fed periodic distance method.
def __cluster_labels__(df, epsilon, min_samples, metric_type = "euclidian", arena_lengths = None):
    
    matrix_data = df[['x_pos','y_pos']].to_numpy()
    arena_size_x = arena_lengths[0]
    arena_size_y = arena_lengths[1]

    if (metric_type == 'euclidian'):
        
        clustering = DBSCAN(epsilon, min_samples, algorithm = 'kd_tree').fit(matrix_data)
    
    elif (metric_type == 'periodic'):
        
        clustering = DBSCAN(epsilon, min_samples, metric= distance_between_periodic_points, metric_params = 
                            {"arena_size_x": arena_size_x, "arena_size_y": arena_size_y}).fit(matrix_data)
        
    return(clustering.labels_)


# adds clusters to the dataframe. returns a dataframe with an additional column of cluster labels.

def add_clusters(df, epsilon, min_samples, metric_type = "euclidian", arena_lengths = None):
    
    df["cluster"] = __cluster_labels__(df, epsilon, min_samples, metric_type, arena_lengths)
    
    return(df)

# returns a dataframe with all individuals in the given cluster 
def get_cluster(df, cluster_num):

    cluster = df[df['cluster'] == cluster_num]

    cluster.reset_index(drop = True, inplace = True)
    
    return(cluster)

# returns a list of dictionaries. Each list index corresponds to a single cluster. The dicionaries contain summary statistics.
def all_cluster_stats(df, precision = 6, periodic = (False, None, None)):

    cluster_nums = pd.unique(df.loc[:,'cluster'])
   
    all_clusters = {}
    
    for i in cluster_nums:
        
        if (i == -1): continue

        cluster = get_cluster(df, int(i))
        cluster_stats = summary_statistics(cluster, periodic = periodic)

        all_clusters[i] = cluster_stats

    return(all_clusters)


# visualize the clusters using open cv. Work in progress. 

def visualize_clusters(df, max_x, max_y, periodic = (False,None,None)):

    
    line_length = 10
    cluster_dict = all_cluster_stats(df, periodic = periodic)
    visualization = np.zeros((max_x,max_y,3), np.uint8)
    
    for cluster_num in cluster_dict: 
        
        if (cluster_num == -1):
            continue
        
        centroid = (cluster_dict[cluster_num].get("centroid"))

        average_displacement = (cluster_dict[cluster_num].get("average_distance"))

        average_dir = (cluster_dict[cluster_num].get("average_dir"))
       
        polarization = (cluster_dict[cluster_num].get("polarization"))
      
        angular_momentum = (cluster_dict[cluster_num].get("angular_momentum"))
        
        number = (cluster_dict[cluster_num].get("number"))
        
    
        pt1 = (int(centroid[0]), int(centroid[1]))

        multiplier = 30 * polarization
        pt2 = (int(pt1[0] + average_dir[0]* multiplier), int(pt1[1] + average_dir[1] * multiplier))
 

        cv2.arrowedLine(visualization, pt1, pt2, (255,255,255), 1)
        cv2.circle(visualization, pt1, number * 2, (0,0,255))

    
    cv2.imshow("group_statistics", visualization)
    
    return (cv2.waitKey(1) & 0xFF == 27)




# returns distance (scalar) between two points in a M x N periodic "arena"
def distance_between_periodic_points(point_x, point_y, arena_size_x, arena_size_y):
    
     # vector between the two positions of i and j 
    temp_vector = point_y - point_x

    # check if closer through periodic boundary and calculate difference
    if abs(temp_vector[0]) > arena_size_x / 2:

        if point_x[0] < point_y[0]:
            temp_vector[0] = (point_y[0] 
                             - (point_x[0] + arena_size_x))
        else:
            temp_vector[0] = (point_y[0]
                             - (point_x[0] - arena_size_x))

    if abs(temp_vector[1]) > arena_size_y / 2:

        if (point_x[1] < point_y[1]):
            temp_vector[1] = (point_y[1] 
                             - (point_x[1] + arena_size_y))
        else:
            temp_vector[1] = (point_y[1] 
                             - (point_x[1] - arena_size_y))
    
    return np.linalg.norm(temp_vector)

# returns displacement (vector) between two points in a M x N periodic "arena"
def vector_between_periodic_points(point_x, point_y, arena_size_x, arena_size_y):

     # vector between the two positions of i and j 
    temp_vector = point_y - point_x

    # check if closer through periodic boundary and calculate difference
    if abs(temp_vector[0]) > arena_size_x / 2:

        if point_x[0] < point_y[0]:
            temp_vector[0] = (point_y[0] 
                             - (point_x[0] + arena_size_x))
        else:
            temp_vector[0] = (point_y[0]
                             - (point_x[0] - arena_size_x))

    if abs(temp_vector[1]) > arena_size_y / 2:

        if (point_x[1] < point_y[1]):
            temp_vector[1] = (point_y[1] 
                             - (point_x[1] + arena_size_y))
        else:
            temp_vector[1] = (point_y[1] 
                             - (point_x[1] - arena_size_y))
    
    return (temp_vector)

# based on https://en.wikipedia.org/wiki/Mean_of_circular_quantities and 
# https://gist.github.com/jonesor/132f531a520c3b331543

def mean_of_circular_vals(data,max_degree):

    array_vals = np.array(data)
    
    # convert to radians
    radians = array_vals * (360 / max_degree) * (math.pi / 180)
    
    meancos = sum(map(math.cos, radians)) / len(radians)
    
    if meancos == 0: return -1
    
    meansin = sum(map(math.sin, radians)) / len(radians)
    
    x = np.arctan(meansin / meancos)

    if  meancos < 0:

        x = x + (math.pi)

    elif meansin * 180 / math.pi < 0: 

        x = x + (2 * math.pi)

    x = x * (180 / math.pi) * (max_degree/360)

    x = round(x, 5)


    return(x)

# test methods
if __name__ == '__main__':
    
    data = {'x_pos': [499, 1, 498, 250,251,100,101,102],
           'y_pos': [1,1, .5,250,251,100,101,102],
           'x_dir': [1,0, 0,0,1,1,1,1], 
            'y_dir' : [0,-1, -1,1,0,0,0,0]}
    
    df = pd.DataFrame(data)
    check_data(df)

    
    df = add_clusters(df, 3, 3, metric_type = "periodic", arena_lengths = [500,500])
    print(df)
    
    print("\ntesting clusters:")
    
    cluster_stats = all_cluster_stats(df, periodic = [True, 500, 500])
    print(cluster_stats)
    print("\n")
    
    
    visualize_clusters(df, 500, 500, periodic = [True, 500, 500])


    
    


    


   x_pos  y_pos  x_dir  y_dir  cluster
0    499    1.0      1      0        0
1      1    1.0      0     -1        0
2    498    0.5      0     -1        0
3    250  250.0      0      1       -1
4    251  251.0      1      0       -1
5    100  100.0      1      0        1
6    101  101.0      1      0        1
7    102  102.0      1      0        1

testing clusters:
{0: {'centroid': (499.33331, 0.83333), 'average_distance': 1.140669, 'average_dir': (0.333333, -0.666667), 'polarization': 0.745356, 'angular_momentum': 0.166683, 'number': 3}, 1: {'centroid': (101.0, 101.0), 'average_distance': 0.942809, 'average_dir': (1.0, 0.0), 'polarization': 1.0, 'angular_momentum': 0.0, 'number': 3}}


